In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install pandas
!pip install numpy

In [49]:
import time
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from transformers import pipeline

In [50]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [51]:
list_of_labels = ['Cardio', 'Chest, Shoulders, Triceps, Neck Exercise', 'Lats, Traps, Lower Back Exercise', 'Biceps, Forearms Exercise', 'Abdominals, Obliques Exercise', 'Quads, Hamstrings, Calves, Glutes Exercise']

In [52]:
# Add your Fitbod File here
df_fitbod = pd.read_csv("WorkoutExport_Fitbod.csv")

In [53]:
# Sample Strong file provided by Hevy
df_strong = pd.read_csv("workout_data_strong.csv")

In [54]:
df_fitbod[:5]

,Date,Exercise,Reps,Weight(kg),Duration(s),Distance(m),Incline,Resistance,isWarmup,Note,multiplier
0,2023-10-07 20:30:10 +0000,Elliptical,0,0.0,1605.0,0.0,0.0,0.0,False,NaN,0.0
1,2023-10-07 18:57:43 +0000,Assisted Wide Grip Pull Up,10,14.0,0.0,0.0,0.0,0.0,False,NaN,0.0
2,2023-10-07 18:57:43 +0000,Assisted Wide Grip Pull Up,10,14.0,0.0,0.0,0.0,0.0,False,NaN,0.0
3,2023-10-07 18:57:43 +0000,Assisted Wide Grip Pull Up,6,14.0,0.0,0.0,0.0,0.0,False,NaN,0.0
4,2023-10-07 18:57:43 +0000,Assisted Wide Grip Pull Up,6,14.0,0.0,0.0,0.0,0.0,False,NaN,0.0


In [55]:
fitbod_exercise = list(set(df_fitbod['Exercise'].to_list()))

In [59]:
len(fitbod_exercise)

136

In [60]:
# exercise_conversion = {'Rowing' : 'Rowing Machine',
#  'Machine Hip Adductor' : 'Hip Adduction (Machine)',
# #  'Leg Pull-In',
#  'Barbell Lunge' : 'Lunge (Barbell)',
#  'Superman Hold': 'Superman',
#  'Stair Stepper' : 'Stair Machine',
#  'Seated Machine Calf Press': 'Seated Calf Raise',
#  'Air Squats' : 'Squat (Bodyweight)',
#  'Front Squat' : 'Front Squat',
# #  'Tricep Stretch',
#  'Machine Shoulder Press': 'Seated Shoulder Press (Machine)',
#  'Incline Dumbbell Curl': 'Seated Incline Curl (Dumbbell)',
#  'Superman with Scaption': 'Superman',
#  'Dumbbell Fly': 'Chest Fly (Dumbbell)',
#  'Crunches': 'Crunch',
#  'Dumbbell Front Raise': 'Front Raise (Dumbbell)',
#  'Medicine Ball Russian Twist': 'Russian Twist (Weighted)',
#  'Barbell Curl': 'Bicep Curl (Barbell)',
# #  'Decline Russian Twists',
#  'Palms-Down Dumbbell Wrist Curl': 'Seated Palms Up Wrist Curl',
#  'Assisted Chin Up': 'Chin Up (Assisted)',
#  'Dumbbell Back Fly': 'Chest Supported Reverse Fly (Dumbbell)',
#  'Dumbbell Incline Fly': 'Incline Chest Fly (Dumbbell)',
#  'Barbell Shoulder Press': 'Overhead Press (Barbell)',
#  'Leg Raise': 'Lying Leg Raise',
#  'Dumbbell Lateral Raise': 'Lateral Raise (Dumbbell)',
#  'Deadlift': 'Deadlift (Barbell)',
#  'Hammerstrength Iso Row': 'Iso-Lateral Row (Machine)',
#  'Wide Grip Lat Pulldown': 'Lat Pulldown (Cable)',
#  'Reverse Crunch': 'Reverse Crunch',
#  'Skullcrusher': 'Skullcrusher (EZ Bar)',
#  'Bicycle Crunch': 'Bicycle Crunch',
#  'Machine Row': 'Iso-Lateral Row (Machine)',
#  'Dumbbell Squat': 'Squat (Dumbbell)',
#  'Cable Crossover Fly': 'Cable Fly Crossovers',
#  'Burpee': 'Burpee',
#  "Farmer's Walk": 'Farmers Walk',
#  'Dumbbell Skullcrusher': 'Skullcrusher (Dumbbell)',
#  'Back Squat': 'Squat (Barbell)',
#  'Dumbbell Lunge': 'Lunge (Dumbbell)',
#  'Face Down Plate Neck Resistance': 'Lying Neck Extension (Weighted)',
#  'Dumbbell Bench Press': 'Bench Press (Dumbbell)',
#  'Machine Overhead Press': 'Seated Shoulder Press (Machine)',
#  'Incline EZ-Bar Curl': 'Incline Curl (EZ Bar)',
#  'Cable Lateral Raise': 'Single Arm Lateral Raise (Cable)',
#  'Hiking': 'Hiking',
#  'Dumbbell Romanian Deadlift': 'Romanian Deadlift (Dumbbell)',
#  'Push Press': 'Push Press',
#  'Back Extensions': 'Back Extension (Weighted Hyperextension)',
#  'EZ-Bar Overhead Tricep Extension': 'Overhead Tricep Extension (EZ Bar)',
#  'Push Up': 'Push Up',
#  'Dumbbell Row': 'Dumbbell Row',
#  'Preacher Curl': 'Preacher Curl (EZ Bar)',
#  'Machine Leg Press': 'Leg Press Horizontal (Machine)',
#  'Dumbbell Tricep Extension': 'Triceps Extension (Dumbbell)',
#  'Leg Extension': 'Leg Extension (Machine)',
#  'Elliptical': 'Elliptical Trainer',
#  'Machine Bicep Curl': 'Preacher Curl (Machine)',
#  'Machine Reverse Fly': 'Rear Delt Reverse Fly (Machine)',
#  'Scissor Crossover Kick': 'Ab Scissors',
#  'Barbell Bench Press': 'Bench Press (Barbell)',
#  'Dead Hang': 'Dead Hang',
#  'Dumbbell Shrug': 'Shrug (Dumbbell)',
#  'Leg Press': 'Inclined Leg Press',
#  'Bent Over Barbell Row': 'Bent Over Row (Barbell)',
#  'Decline Push Up': 'Decline Push Up',
#  'Romanian Deadlift': 'Romanian Deadlift (Barbell)',
#  'Cable Tricep Pushdown': 'Triceps Pushdown',
#  'Cable Rope Tricep Extension': 'Triceps Rope Pushdown',
#  'Seated Dumbbell Curl': 'Bicep Curl (Dumbbell)',
#  'Incline Hammer Curl': 'Incline Hammer Curl (Dumbbell)',
#  'Seated Tricep Press': 'Seated Triceps Press',
#  'Palms-Up Dumbbell Wrist Curl': 'Seated Palms Up Wrist Curl',
#  'Dead Bug': 'Dead Bug',
#  'Tricep Extension': 'Triceps Extension (Dumbbell)',
#  'Hammerstrength Chest Press': 'Chest Press (Machine)',
#  'Hammer Curls': 'Hammer Curl (Dumbbell)',
#  'Assisted Dip': 'Triceps Dip (Assisted)',
#  'Russian Twist': 'Russian Twist (Weighted)',
#  'Dumbbell Upright Row': 'Upright Row (Dumbbell)',
#  'Dumbbell Squat To Shoulder Press': 'Shoulder Press (Dumbbell)',
#  'Reverse Barbell Curl': 'Reverse Curl (Barbell)',
#  'Cycling - Stationary': 'Spinning',
#  'Running': 'Running',
#  'Cable Face Pull': 'Face Pull',
#  'Cable Row': 'Seated Cable Row - V Grip (Cable)',
#  'Decline Crunch': 'Decline Crunch (Weighted)',
#  'Superman': 'Superman',
#  'Reverse Dumbbell Curl': 'Reverse Curl (Dumbbell)',
#  'Stiff-Legged Barbell Good Morning': 'Good Morning (Barbell)',
#  'Machine Hip Abductor': 'Hip Abduction (Machine)',
#  'Hack Squat': 'Hack Squat (Machine)',
#  'Dumbbell Kickbacks': 'Triceps Kickback (Dumbbell)',
#  'Walking': 'Walking',
#  'Dumbbell Bicep Curl': 'Bicep Curl (Dumbbell)',
#  'Assisted Pull Up': 'Pull Up (Assisted)',
#  'Lying Hamstrings Curl': 'Lying Leg Curl (Machine)',
#  'Scissor Kick': 'Ab Scissors',
#  'Cycling': 'Cycling',
#  'Dumbbell Shoulder Raise': 'Lateral Raise (Dumbbell)',
#  'Cable One Arm Underhand Tricep Extension': 'Single Arm Triceps Pushdown (Cable)',
#  'Lat Pulldown': 'Lat Pulldown (Cable)',
#  'Assisted Wide Grip Pull Up': 'Pull Up (Assisted)',
#  'Hammerstrength Incline Chest Press': 'Incline Chest Press (Machine)',
#  'T-Bar Row': 'T Bar Row',
#  'Barbell Incline Bench Press': 'Incline Bench Press (Barbell)',
#  'Plank': 'Plank',
#  'Close-Grip Bench Press': 'Bench Press - Close Grip (Barbell)',
#  'Standing Leg Side Hold': 'Single Leg Standing Calf Raise',
#  'Seated Leg Curl': 'Seated Leg Curl (Machine)',
#  'Ab Crunch Machine': 'Crunch (Machine)',
#  'Kettlebell Swing': 'Kettlebell Swing',
#  'Dumbbell Shoulder Press': 'Shoulder Press (Dumbbell)',
#  'Dumbbell Incline Bench Press': 'Incline Bench Press (Dumbbell)',
#  'Dumbbell Rear Delt Raise': 'Rear Delt Reverse Fly (Dumbbell)',
#  'Sit Up': 'Sit Up',
#  'Incline Dumbbell Row': 'Chest Supported Incline Row (Dumbbell)',
#  'Hammerstrength Decline Chest Press': 'Decline Bench Press (Machine)',
#  'Seated Back Extension': 'Back Extension (Machine)',
#  'Glute Kickback Machine': 'Glute Kickback (Machine)',
#  'EZ-Bar Curl': 'EZ Bar Biceps Curl',
# #  'Backward Arm Circle',
#  'Upright Row': 'Upright Row (Barbell)',
#  'Hammerstrength Shoulder Press': 'Shoulder Press (Machine Plates)',
#  'Standing Machine Calf Press': 'Standing Calf Raise (Machine)',
#  'Leg Curl': 'Seated Leg Curl (Machine)',
#  'Barbell Hip Thrust': 'Hip Thrust Machine',
#  'Side Bridge': 'Side Plank',
#  'Arnold Dumbbell Press': 'Arnold Press (Dumbbell)',
#  'Dumbbell Decline Bench Press': 'Decline Bench Press (Dumbbell)',
#  'Zottman Curl': 'Zottman Curl (Dumbbell)',
#  'Barbell Shrug': 'Shrug (Barbell)',
#  'Running - Treadmill': 'Treadmill',
#  'Pull Up': 'Pull Up', 'Machine Tricep Dip' : 'Seated Dip Machine',
#            'Smith Machine Squat': 'Squat (Smith Machine)'}

In [61]:
df_fitbod[:3]

,Date,Exercise,Reps,Weight(kg),Duration(s),Distance(m),Incline,Resistance,isWarmup,Note,multiplier
0,2023-10-07 20:30:10 +0000,Elliptical,0,0.0,1605.0,0.0,0.0,0.0,False,NaN,0.0
1,2023-10-07 18:57:43 +0000,Assisted Wide Grip Pull Up,10,14.0,0.0,0.0,0.0,0.0,False,NaN,0.0
2,2023-10-07 18:57:43 +0000,Assisted Wide Grip Pull Up,10,14.0,0.0,0.0,0.0,0.0,False,NaN,0.0


In [62]:
# for index, row in df_fitbod.iterrows():
#     if row['Exercise'] in exercise_conversion:
#         df_fitbod.at[index,'Exercise'] = exercise_conversion[row['Exercise']]

In [63]:
# exercise_title_label = dict()
# for index, row in df_fitbod.iterrows():
#     if row['Exercise'] not in exercise_title_label:
#         exercise_title_label[row['Exercise']] = classifier(row['Exercise'], list_of_labels)['labels'][0] + " - At Gym"

In [64]:
# with open('exercise_title_label.json', 'w', encoding ='utf8') as f:
#     json.dump(exercise_title_label, f, ensure_ascii = False)

In [65]:
# with open('exercise_title_label.json', 'r', encoding='utf-8') as json_file:
#     exercise_title_label = json.load(json_file)

In [66]:
df_fitbod[:4]

,Date,Exercise,Reps,Weight(kg),Duration(s),Distance(m),Incline,Resistance,isWarmup,Note,multiplier
0,2023-10-07 20:30:10 +0000,Elliptical,0,0.0,1605.0,0.0,0.0,0.0,False,NaN,0.0
1,2023-10-07 18:57:43 +0000,Assisted Wide Grip Pull Up,10,14.0,0.0,0.0,0.0,0.0,False,NaN,0.0
2,2023-10-07 18:57:43 +0000,Assisted Wide Grip Pull Up,10,14.0,0.0,0.0,0.0,0.0,False,NaN,0.0
3,2023-10-07 18:57:43 +0000,Assisted Wide Grip Pull Up,6,14.0,0.0,0.0,0.0,0.0,False,NaN,0.0


In [67]:
# strong_cols = df_strong.columns.to_list()

tilte - zero shot learning ['Chest Workout', 'Shoulder/ Triceps Workout', 'Back Workout', 'Biceps / Forearm Workout', 'Neck Workout', 'Legs Workout'] \
start_time - date \
end_time - date \
description  - None \
exerecise_title - exercise \
superset_id - None \
exercise_notes - None \
set_index - to be calculated \
set_type - 'normal' \
weight_lbs - covert to kgs \
reps - Reps \
distance_km - distance(m) \
duration_seconds - Duration(s)

In [72]:
df_fitbod = df_fitbod.sort_values(by=['Date', 'Exercise'], ascending=[True, True])

In [73]:
len(df_fitbod)

4172

In [74]:
df_strong[:5]

,title,start_time,end_time,description,exercise_title,superset_id,exercise_notes,set_index,set_type,weight_lbs,reps,distance_km,duration_seconds
0,Shoulders and Core - At home,"29 Jan 2022, 15:15","29 Jan 2022, 16:23",NaN,Roll Out Shoulder,NaN,NaN,0,normal,NaN,NaN,NaN,120.0
1,Shoulders and Core - At home,"29 Jan 2022, 15:15","29 Jan 2022, 16:23",NaN,Roll Out Shoulder,NaN,NaN,1,normal,NaN,NaN,NaN,120.0
2,Shoulders and Core - At home,"29 Jan 2022, 15:15","29 Jan 2022, 16:23",NaN,Roll Out Shoulder,NaN,NaN,2,normal,NaN,NaN,NaN,120.0
3,Shoulders and Core - At home,"29 Jan 2022, 15:15","29 Jan 2022, 16:23",NaN,Roll Out Shoulder,NaN,NaN,3,normal,NaN,NaN,NaN,120.0
4,Shoulders and Core - At home,"29 Jan 2022, 15:15","29 Jan 2022, 16:23",NaN,Floss Shoulders,NaN,NaN,0,normal,NaN,15.0,NaN,NaN


In [75]:
fitbod_to_strong_df = pd.DataFrame(columns=df_strong.columns)
# fitbod_to_strong_df = fitbod_to_strong_df.astype(df_strong.dtypes.to_dict())

In [76]:
df_strong.dtypes

title                object
start_time           object
end_time             object
description          object
exercise_title       object
superset_id         float64
exercise_notes       object
set_index             int64
set_type             object
weight_lbs          float64
reps                float64
distance_km         float64
duration_seconds    float64
dtype: object

In [77]:
fitbod_to_strong_df.dtypes

title               object
start_time          object
end_time            object
description         object
exercise_title      object
superset_id         object
exercise_notes      object
set_index           object
set_type            object
weight_lbs          object
reps                object
distance_km         object
duration_seconds    object
dtype: object

In [78]:
## Change the Exercise Names matching to Hevy

In [79]:
# fitbod_exercise_name = set(df_fitbod['Exercise'].to_list())

In [80]:
# exercise = {'Leg Press' : 'Inclined Leg Press',
#            'Barbell Hip Thrust' : 'Hip Thrust Machine'}

In [81]:
# for key, val in exercise.items():
#     condition = df_fitbod['Exercise'] == key
#     df_fitbod.loc[condition, 'Exercise'] = val

In [82]:
startTime = time.time()
title = list()
start_time = list()
end_time = list()
description = list()
exercise_title = list()
superset_id = list()
exercise_notes = list()
set_index = list()
set_type = list()
weight_lbs = list()
reps = list()
distance_km = list()
duration_seconds = list()
_key = ''
_title = ''
for index, row in df_fitbod[:50].iterrows():
    key = row['Exercise'] + str(row['Date'])
    timestamp = datetime.strptime(row['Date'], '%Y-%m-%d %H:%M:%S %z')
    if float(row['Duration(s)']) > 0.0:
        duration_seconds.append(float(row['Duration(s)']))
        end_timestamp = timestamp + timedelta(seconds=int(row['Duration(s)']))
    else:
        duration_seconds.append(float(60))
        end_timestamp = timestamp + timedelta(seconds=60)
        
    print(timestamp, end_timestamp, row['Duration(s)'])
    
    formatted_timestamp = timestamp.strftime('%d %b %Y, %H:%M')
    formatted_end_timestamp = end_timestamp.strftime('%d %b %Y, %H:%M')
    
    start_time.append(formatted_timestamp)
    end_time.append(formatted_end_timestamp)
    description.append(np.nan)
    exercise_title.append(row['Exercise'])
    superset_id.append(np.nan)
    exercise_notes.append(np.nan)
    if key != _key:
        count = 0
        _key = key
        _title = classifier(row['Exercise'], list_of_labels)['labels'][0] + " - At Gym"
        title.append(_title)
    elif key == _key:
        count += 1
        title.append(_title)

    set_index.append(int(count))
    set_type.append('normal')
    weight_lbs.append(round(float(row['Weight(kg)'] * 2.20462), 2))
    reps.append(float(row['Reps']))
    distance_km.append(round(float(row['Distance(m)'] / 1000), 2))
    
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

2023-01-15 18:58:11+00:00 2023-01-15 19:10:58+00:00 767.0
2023-01-18 18:45:50+00:00 2023-01-18 19:00:44+00:00 894.0
2023-01-19 19:09:24+00:00 2023-01-19 19:32:26+00:00 1382.0
2023-01-19 19:24:17+00:00 2023-01-19 19:25:17+00:00 0.0
2023-01-19 19:24:17+00:00 2023-01-19 19:25:17+00:00 0.0
2023-01-19 19:24:17+00:00 2023-01-19 19:25:17+00:00 0.0
2023-01-19 19:24:17+00:00 2023-01-19 19:25:17+00:00 0.0
2023-01-19 19:24:17+00:00 2023-01-19 19:25:17+00:00 0.0
2023-01-19 19:24:17+00:00 2023-01-19 19:25:17+00:00 0.0
2023-01-19 19:24:17+00:00 2023-01-19 19:25:17+00:00 0.0
2023-01-19 19:24:17+00:00 2023-01-19 19:25:17+00:00 0.0
2023-01-19 19:24:17+00:00 2023-01-19 19:25:17+00:00 0.0
2023-01-19 19:24:17+00:00 2023-01-19 19:25:17+00:00 0.0
2023-01-19 19:24:17+00:00 2023-01-19 19:25:17+00:00 0.0
2023-01-19 19:24:17+00:00 2023-01-19 19:25:17+00:00 0.0
2023-01-19 19:24:17+00:00 2023-01-19 19:25:17+00:00 0.0
2023-01-19 19:24:17+00:00 2023-01-19 19:25:17+00:00 0.0
2023-01-19 19:24:17+00:00 2023-01-19 19:2

In [89]:
fitbod_to_strong_df['title'] = title
fitbod_to_strong_df['start_time'] = start_time
fitbod_to_strong_df['end_time'] = end_time
fitbod_to_strong_df['description'] = description
fitbod_to_strong_df['exercise_title'] = exercise_title
fitbod_to_strong_df['superset_id'] = superset_id
fitbod_to_strong_df['exercise_notes'] = exercise_notes
fitbod_to_strong_df['set_index'] = set_index
fitbod_to_strong_df['set_type'] = set_type
fitbod_to_strong_df['weight_lbs'] = weight_lbs
fitbod_to_strong_df['reps'] = reps
fitbod_to_strong_df['distance_km'] = distance_km
fitbod_to_strong_df['duration_seconds'] = duration_seconds

In [90]:
fitbod_to_strong_df['reps'] = np.where(fitbod_to_strong_df['reps'] == 0.0, np.nan, fitbod_to_strong_df['reps'])

In [91]:
fitbod_to_strong_df['description'] = fitbod_to_strong_df['description'].astype(object)
fitbod_to_strong_df['exercise_notes'] = fitbod_to_strong_df['exercise_notes'].astype(object)

In [94]:
df_strong.dtypes # Strong Columns and Datatypes

title                object
start_time           object
end_time             object
description          object
exercise_title       object
superset_id         float64
exercise_notes       object
set_index             int64
set_type             object
weight_lbs          float64
reps                float64
distance_km         float64
duration_seconds    float64
dtype: object

In [95]:
fitbod_to_strong_df.dtypes # Converted Columns and Datatypes

title                object
start_time           object
end_time             object
description          object
exercise_title       object
superset_id         float64
exercise_notes       object
set_index             int64
set_type             object
weight_lbs          float64
reps                float64
distance_km         float64
duration_seconds    float64
dtype: object

In [98]:
fitbod_to_strong_df.to_csv("fitbod_to_strong_workout.csv", index=False)